In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

import os

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

# Y_train_8 = pd.read_csv('Data/Y_train_8.csv', header=None, delimiter=',').values
# Y_test_8 = pd.read_csv('Data/Y_test_8.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('Data/Y_test_std_21.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values


train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

# train_y_8 = torch.tensor(Y_train_8, dtype=torch.float32)
# test_y_8 = torch.tensor(Y_test_8, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)

Device = 'cpu'

In [ ]:
num_latents_list = [12, 14, 16, 18]
num_inducing_list = [100, 150, 175, 200, 300, 400]
covar_type_list = ['Matern5/2', 'RBF', 'RQ']

best_mse = float('inf')
best_params = None
results = []  # 存储所有成功组合及对应的 mse

for num_latents in num_latents_list:
    for num_inducing in num_inducing_list:
        for covar_type in covar_type_list:
            try:
                # 训练模型
                MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(
                    train_x, train_y_21,
                    covar_type = covar_type,
                    num_latents=num_latents,
                    num_inducing=num_inducing,
                    lr_hyper=0.05,
                    lr_variational=0.05,
                    num_iterations=5000,
                    patience=50,
                    device=Device
                )
                # 得到预测值
                full_test_preds_MVGP = Prediction.preds_for_one_model(
                    MVGP_models, MVGP_likelihoods, test_x.to(Device)
                ).cpu().detach().numpy()

                # 计算均方误差
                mse = np.mean((full_test_preds_MVGP.reshape(120, 21) - test_y_21.numpy()) ** 2)
                results.append((num_latents, num_inducing, mse))

                if mse < best_mse:
                    best_mse = mse
                    best_params = (num_latents, num_inducing)
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} -> mse: {mse}")

            except Exception as e:
                # 出现异常，记录并跳过当前参数组合
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} 训练失败: {e}")
                continue

print("最佳 MSE:", best_mse)
print("最佳参数组合 (num_latents, num_inducing):", best_params)

 39%|███▊      | 1928/5000 [02:02<03:15, 15.71it/s, loss=-18.1]


组合 num_latents=12, num_inducing=100, covar_type=Matern5/2 -> mse: 0.0103071890771389


 22%|██▏       | 1086/5000 [01:01<03:42, 17.59it/s, loss=-17.4]  


组合 num_latents=12, num_inducing=100, covar_type=RBF -> mse: 0.0111930500715971


 28%|██▊       | 1385/5000 [01:33<04:05, 14.75it/s, loss=-18.3] 


...

组合 num_latents=12, num_inducing=150, covar_type=RBF -> mse: 0.010283540934324265


 38%|███▊      | 1916/5000 [02:42<04:22, 11.76it/s, loss=-19.2]


组合 num_latents=12, num_inducing=150, covar_type=RQ -> mse: 0.01022344920784235


  0%|          | 0/5000 [00:00<?, ?it/s]


Intel MKL ERROR: Parameter 6 was incorrect on entry to SLASWP.


In [2]:
num_latents_list = [14, 16, 18]
num_inducing_list = [100, 150]
covar_type_list = ['Matern5/2']

best_mse = float('inf')
best_params = None
results = []  # 存储所有成功组合及对应的 mse

for num_latents in num_latents_list:
    for num_inducing in num_inducing_list:
        for covar_type in covar_type_list:
            try:
                # 训练模型
                MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(
                    train_x, train_y_21,
                    covar_type = covar_type,
                    num_latents=num_latents,
                    num_inducing=num_inducing,
                    lr_hyper=0.05,
                    lr_variational=0.05,
                    num_iterations=5000,
                    patience=50,
                    device=Device
                )
                # 得到预测值
                full_test_preds_MVGP = Prediction.preds_for_one_model(
                    MVGP_models, MVGP_likelihoods, test_x.to(Device)
                ).cpu().detach().numpy()

                # 计算均方误差
                mse = np.mean((full_test_preds_MVGP.reshape(120, 21) - test_y_21.numpy()) ** 2)
                results.append((num_latents, num_inducing, mse))

                if mse < best_mse:
                    best_mse = mse
                    best_params = (num_latents, num_inducing)
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} -> mse: {mse}")

            except Exception as e:
                # 出现异常，记录并跳过当前参数组合
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} 训练失败: {e}")
                continue

print("最佳 MSE:", best_mse)
print("最佳参数组合 (num_latents, num_inducing):", best_params)

 25%|██▌       | 1269/5000 [01:29<04:23, 14.16it/s, loss=-19.4] 


组合 num_latents=14, num_inducing=100, covar_type=Matern5/2 -> mse: 0.007097904104739428


 36%|███▌      | 1805/5000 [08:27<14:57,  3.56it/s, loss=-21.3]


组合 num_latents=14, num_inducing=150, covar_type=Matern5/2 -> mse: 0.006581886205822229


 28%|██▊       | 1403/5000 [03:07<07:59,  7.50it/s, loss=-23.2]


...

组合 num_latents=16, num_inducing=150, covar_type=Matern5/2 -> mse: 0.003858415177091956


 41%|████▏     | 2071/5000 [05:22<07:36,  6.42it/s, loss=-26]  


组合 num_latents=18, num_inducing=100, covar_type=Matern5/2 -> mse: 0.004261645954102278


 29%|██▊       | 1429/5000 [12:12<30:29,  1.95it/s, loss=-27.5]

组合 num_latents=18, num_inducing=150, covar_type=Matern5/2 -> mse: 0.0023993318900465965
最佳 MSE: 0.002399332
最佳参数组合 (num_latents, num_inducing): (18, 150)


In [2]:
num_latents_list = [14, 16, 18, 20]
num_inducing_list = [100, 150]
covar_type_list = ['RBF']

best_mse = float('inf')
best_params = None
results = []  # 存储所有成功组合及对应的 mse

for num_latents in num_latents_list:
    for num_inducing in num_inducing_list:
        for covar_type in covar_type_list:
            try:
                # 训练模型
                MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(
                    train_x, train_y_21,
                    covar_type = covar_type,
                    num_latents=num_latents,
                    num_inducing=num_inducing,
                    lr_hyper=0.05,
                    lr_variational=0.05,
                    num_iterations=5000,
                    patience=50,
                    device=Device
                )
                # 得到预测值
                full_test_preds_MVGP = Prediction.preds_for_one_model(
                    MVGP_models, MVGP_likelihoods, test_x.to(Device)
                ).cpu().detach().numpy()

                # 计算均方误差
                mse = np.mean((full_test_preds_MVGP.reshape(120, 21) - test_y_21.numpy()) ** 2)
                results.append((num_latents, num_inducing, mse))

                if mse < best_mse:
                    best_mse = mse
                    best_params = (num_latents, num_inducing)
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} -> mse: {mse}")

            except Exception as e:
                # 出现异常，记录并跳过当前参数组合
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} 训练失败: {e}")
                continue

print("最佳 MSE:", best_mse)
print("最佳参数组合 (num_latents, num_inducing):", best_params)

 48%|████▊     | 2389/5000 [04:28<04:53,  8.90it/s, loss=-21.9] 


组合 num_latents=14, num_inducing=100, covar_type=RBF -> mse: 0.006956592667847872


 42%|████▏     | 2105/5000 [06:23<08:47,  5.49it/s, loss=-20.1]


组合 num_latents=14, num_inducing=150, covar_type=RBF -> mse: 0.006726214196532965


 40%|███▉      | 1993/5000 [04:07<06:13,  8.05it/s, loss=-22]  


...

组合 num_latents=18, num_inducing=150, covar_type=RBF -> mse: 0.003199865808710456


 28%|██▊       | 1404/5000 [04:58<12:43,  4.71it/s, loss=-28.2]


组合 num_latents=20, num_inducing=100, covar_type=RBF -> mse: 0.0033096533734351397


 27%|██▋       | 1363/5000 [08:53<23:43,  2.55it/s, loss=-32]  

组合 num_latents=20, num_inducing=150, covar_type=RBF -> mse: 0.0020090423058718443
最佳 MSE: 0.0020090423
最佳参数组合 (num_latents, num_inducing): (20, 150)


In [3]:
num_latents_list = [14, 16, 18, 20]
num_inducing_list = [100, 150]
covar_type_list = ['RQ']

best_mse = float('inf')
best_params = None
results = []  # 存储所有成功组合及对应的 mse

for num_latents in num_latents_list:
    for num_inducing in num_inducing_list:
        for covar_type in covar_type_list:
            try:
                # 训练模型
                MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(
                    train_x, train_y_21,
                    covar_type = covar_type,
                    num_latents=num_latents,
                    num_inducing=num_inducing,
                    lr_hyper=0.05,
                    lr_variational=0.05,
                    num_iterations=5000,
                    patience=50,
                    device=Device
                )
                # 得到预测值
                full_test_preds_MVGP = Prediction.preds_for_one_model(
                    MVGP_models, MVGP_likelihoods, test_x.to(Device)
                ).cpu().detach().numpy()

                # 计算均方误差
                mse = np.mean((full_test_preds_MVGP.reshape(120, 21) - test_y_21.numpy()) ** 2)
                results.append((num_latents, num_inducing, mse))

                if mse < best_mse:
                    best_mse = mse
                    best_params = (num_latents, num_inducing)
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} -> mse: {mse}")

            except Exception as e:
                # 出现异常，记录并跳过当前参数组合
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} 训练失败: {e}")
                continue

print("最佳 MSE:", best_mse)
print("最佳参数组合 (num_latents, num_inducing):", best_params)

 46%|████▌     | 2298/5000 [04:55<05:47,  7.77it/s, loss=-21.2] 


组合 num_latents=14, num_inducing=100, covar_type=RQ -> mse: 0.007499956525862217


 35%|███▌      | 1768/5000 [07:42<14:05,  3.82it/s, loss=-21.9]


组合 num_latents=14, num_inducing=150, covar_type=RQ -> mse: 0.007253176067024469


 42%|████▏     | 2119/5000 [05:03<06:52,  6.99it/s, loss=-24]  


...

组合 num_latents=18, num_inducing=150, covar_type=RQ -> mse: 0.0038441182114183903


 50%|████▉     | 2494/5000 [10:02<10:05,  4.14it/s, loss=-28.2] 


组合 num_latents=20, num_inducing=100, covar_type=RQ -> mse: 0.0031142178922891617


 39%|███▊      | 1928/5000 [14:59<23:52,  2.14it/s, loss=-31.3]

组合 num_latents=20, num_inducing=150, covar_type=RQ -> mse: 0.0024792870972305536
最佳 MSE: 0.002479287
最佳参数组合 (num_latents, num_inducing): (20, 150)


In [ ]:
num_latents_list = [22, 24, 26]
num_inducing_list = [150]
covar_type_list = ['RQ']

best_mse = float('inf')
best_params = None
results = []  # 存储所有成功组合及对应的 mse

for num_latents in num_latents_list:
    for num_inducing in num_inducing_list:
        for covar_type in covar_type_list:
            try:
                # 训练模型
                MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(
                    train_x, train_y_21,
                    covar_type = covar_type,
                    num_latents=num_latents,
                    num_inducing=num_inducing,
                    lr_hyper=0.05,
                    lr_variational=0.05,
                    num_iterations=5000,
                    patience=50,
                    device=Device
                )
                # 得到预测值
                full_test_preds_MVGP = Prediction.preds_for_one_model(
                    MVGP_models, MVGP_likelihoods, test_x.to(Device)
                ).cpu().detach().numpy()

                # 计算均方误差
                mse = np.mean((full_test_preds_MVGP.reshape(120, 21) - test_y_21.numpy()) ** 2)
                results.append((num_latents, num_inducing, mse))

                if mse < best_mse:
                    best_mse = mse
                    best_params = (num_latents, num_inducing)
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} -> mse: {mse}")

            except Exception as e:
                # 出现异常，记录并跳过当前参数组合
                print(f"组合 num_latents={num_latents}, num_inducing={num_inducing}, covar_type={covar_type} 训练失败: {e}")
                continue

print("最佳 MSE:", best_mse)
print("最佳参数组合 (num_latents, num_inducing):", best_params)

 21%|██        | 1059/5000 [08:05<30:05,  2.18it/s, loss=-29.7]


组合 num_latents=20, num_inducing=150, covar_type=RQ -> mse: 0.002101204125210643


  0%|          | 0/5000 [00:00<?, ?it/s]


Intel MKL ERROR: Parameter 6 was incorrect on entry to SLASWP.
